In [14]:
%load_ext lab_black
import pandas as pd
import json


f = open("histoday_BTC.json",)
data = json.load(f)
data = pd.DataFrame.from_dict(data["Data"]["Data"])
data = data[["time", "close"]]
f.close()
data

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


,time,close
0,1446076800,313.63
1,1446163200,327.12
2,1446249600,311.24
3,1446336000,322.95
4,1446422400,359.28
...,...,...
1996,1618531200,61416.41
1997,1618617600,60056.07
1998,1618704000,56246.82
1999,1618790400,55680.11


In [12]:
def get_profit(lower_limit, upper_limit):
    i = 0
    profit = 0
    records = []
    n = 0
    for index, row in data.iterrows():
        day = index
        price = row["close"]
        if i % 2 == 0:
            if row["RSI"] <= lower_limit and n == 0:
                profit -= price
                records.append(["Buying", -round(price), round(profit), day])
                n += 1
                i += 1
        else:
            if row["RSI"] >= upper_limit and n >= 0 and profit + price >= 1:
                profit += price
                records.append(["Selling", round(price), round(profit), day])
                n -= 1
                i += 1
    return profit


def calculation(data, window_length, lower_limit, upper_limit):
    delta = data["close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up <= 1] = 0
    down[down >= 2] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS1 = roll_up1 / roll_down1
    RSI1 = 100.0 - (100.0 / (1.0 + RS1))
    data.loc[:, "RSI"] = RSI1
    data.loc[:, "TimeStamp"] = pd.to_datetime(data["time"], unit="s")
    data = data.set_index("TimeStamp")
    return get_profit(lower_limit, upper_limit)


calculation(data, 14, 50, 60)

-55991.28

In [13]:
%%time
from tqdm import tqdm
optimizations=[]
for window_length in tqdm(range(10, 20)):
    for lower_limit in range(10, 30):
        for upper_limit in range(80, 100):
            profit = calculation(data, window_length, lower_limit,upper_limit)
            optimizations.append([profit, window_length, lower_limit, upper_limit])

100%|██████████| 10/10 [26:23<00:00, 158.39s/it]

Wall time: 26min 23s


In [15]:
optimization_df = pd.DataFrame(
    optimizations, columns=["Profit", "WindowLength", "LowerLimit", "UpperLimit"]
)
pd.set_option("max_rows", 30)
optimization_df

,Profit,WindowLength,LowerLimit,UpperLimit
0,2299.81,10,10,80
1,624.36,10,10,81
2,682.07,10,10,82
3,709.60,10,10,83
4,709.60,10,10,84
...,...,...,...,...
3995,-55600.29,19,29,95
3996,-55600.29,19,29,96
3997,-55600.29,19,29,97
3998,-55600.29,19,29,98


In [16]:
# optimization_df.to_excel("Profit_Optimization_BTC_min.xlsx")

In [17]:
optimization_df[optimization_df["Profit"] == optimization_df["Profit"].max()]

,Profit,WindowLength,LowerLimit,UpperLimit
1262,2725.88,13,13,82


In [18]:
window_length = 13
lower_limit = 13
upper_limit = 82

In [20]:
def get_profit(lower_limit, upper_limit):
    i = 0
    profit = 0
    records = []
    n = 0
    for index, row in data.iterrows():
        day = index
        price = row["close"]
        if i % 2 == 0:
            if row["RSI"] <= lower_limit and n == 0:
                profit -= price
                records.append(["Buying", -round(price), round(profit), day])
                n += 1
                i += 1
        else:
            if row["RSI"] >= upper_limit and n >= 0 and profit + price >= 1:
                profit += price
                records.append(["Selling", round(price), round(profit), day])
                n -= 1
                i += 1
    return records, profit


def calculation(data, window_length, lower_limit, upper_limit):
    delta = data["close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up <= 1] = 0
    down[down >= 2] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS1 = roll_up1 / roll_down1
    RSI1 = 100.0 - (100.0 / (1.0 + RS1))
    data.loc[:, "RSI"] = RSI1
    data.loc[:, "TimeStamp"] = pd.to_datetime(data["time"], unit="s")
    data = data.set_index("TimeStamp")
    records, profit = get_profit(lower_limit, upper_limit)
    return records, profit


records, profit = calculation(data, window_length, lower_limit, upper_limit)
formated_profit = f"{round(profit):,}"
print(f"Total profit: USD {formated_profit}\n")
records_df = pd.DataFrame(records, columns=["Action", "Price", "Profit", "Day"])
pd.set_option("max_rows", 30)
records_df

Total profit: USD 5,601



,Action,Price,Profit,Day
0,Buying,-607,-607,277
1,Selling,626,19,315
2,Buying,-5741,-5723,1112
3,Selling,5840,117,1283
4,Buying,-8543,-8425,1426
5,Selling,10158,1732,1564
6,Buying,-4917,-3184,1596
7,Selling,8785,5601,1644
